# Introduction

## Explanation 1.1: Linear Programming

In the offline scenario, the total utility of the optimal task assignment, which is marked in red in TABLE 1, is 17.

|    | t1 | t2 | t3 | t4 | t5 | t6 |
|----|----|----|----|----|----|----|
| w1(1) |  7  |  1   |    |    |    |    |
| w2(3) |    |     |  1  |    |    |    |
| w3(2) |    |    |   9 |  1  |  1  |  1  |

In [75]:
from scipy.optimize import linprog
import numpy as np

c = -np.array([7, 1, 0, 0, 0, 0,
               0, 0, 1, 0, 0, 0,
               0, 0, 9, 1, 1, 1
               ])

A_ub = [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        ]

b_ub = [1, 3, 2, 1, 1, 1, 1, 1, 1]

bounds = [(0, 1) for _ in range(18)]

res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=1)

res.fun

-17.0

In [76]:
res.x

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  1.])

## Explanation 1.2

The GOMA problem, which generalizes the OMWBM problem, mainly differs from the OMWBM problem in that both the tasks and the workers are dynamic. 

| Timestamp | 0  | 1  | 2 | 7 | 8 | 9 | 9 | 15 | 18 |
|-----------|----|----|---|---|---|---|---|----|----|
| 1st order | t1 | t2 | w1  | w2  | t3  | t4  | w3  | t5   | t6   |
| 2nd order | w1 | t1   | t2  | t3  | w3  | t4  | w2  | t6   | t5   |

see Keynote.

# Problem Statement

Dataset Format Explanation:

## Header

| workerN/n | taskN/m  | Umax  | sumC |
|-----------|----|----|---|
| 817 | 4036 | 10.000000 | 4853  | 

## Task

| begTime | stype  | loc.first  | loc.second | endTime | cost.pay |
|-----------|----|----|---|----|---|
| 74875 | t | 1.790979 | 1.725669  |  600 | 3.6|


## Worker

| begTime | stype  | loc.first  | loc.second | rad | cap | endTime | cost.rate |
|-----------|----|----|---|---|---|---|---|
| 444046 | w |1.230049 |1.007416 |1.000000| 1| 600 |0.597888|

In [88]:
from typing import Tuple, Self
import os
import math
import random


class Vertex:
    """Either a task or a worker. """

    def __init__(self, arrival_time: int, location: Tuple[float], deadline: int, capacity: int) -> None:
        """Initialize a Vertex object."""
        assert len(location) == 2
        self.arrival_time = arrival_time
        self.location = location
        self.deadline = deadline

        # Without loss of generality, Say task also has capacity.
        # 1. The number of tasks assigned to a worker w cannot exceed his/her capacity cw.
        # 2. The number of workers assigned to a task t cannot exceed its capacity (1 here).
        self.capacity = capacity

        # 1. Number of tasks a Worker is assigned to (for Worker)
        # 2. Number of times a Task has been assigned (for Task), initially 0
        self.flow = 0

    def distance_from(self, v: Self) -> float:
        """Calculate the Euclidean distance between self and another vertex. """
        x1, y1 = self.location
        x2, y2 = v.location
        return ((x1-x2)**2+(y1-y2)**2)**(1/2)


class Worker(Vertex):
    """A worker is a tuple <L, A, D, R, C, Delta>, who arrives at the initial location L in the 2D space, 
    at time A. He/She can perform several tasks that arrive at the platform before his/her response deadline D
    with several constraints:
    1. He/She can perform at most C tasks (i.e., his/her capacity); 
    2. He/She can only perform tasks posted within a circular range centered at L and with a radius of R;
    3. Delta is the success ratio based on his/her historical task completion records.
    """

    def __init__(self, arrival_time: int, location: Tuple[float], radius: float, capacity: int, deadline: int, success_ratio: float) -> None:
        """Initializes a Worker object. """
        assert 0 < success_ratio <= 1
        super().__init__(arrival_time, location, deadline, capacity)
        self.radius = radius
        self.capacity = capacity
        self.success_ratio = success_ratio

    def __str__(self) -> str:
        """Returns a string representation of the worker."""
        return f"w: loc = ({self.location[0]:.2f}, {self.location[1]:.2f}), rad = {self.radius:.2f}, cap = {self.capacity}, time = ({self.arrival_time}, {self.deadline}), success_ratio = {self.success_ratio:.2f}"

    def utility_with(self, task: type[Task]) -> float:
        """The utility that a worker performs a task is measured by P_t * Delta_w.
        Without loss of generality, we will assume that utility is between 0 and U_max.
        """
        return task.utility_with(self)

    def can_be_with(self, task: type[Task]) -> float:
        """Check the 4 constraints, namely:
        Invariable Constraint, Range Constraint, Deadline Constraint, Capacity Constraint
        """
        return task.utility_with(self)


class Task(Vertex):
    """A task is a tuple <L, A, D, P>, which is posted at the location L in 2D space, at time A. 
    It can only be allocated to a worker who arrives before the response deadline D, with a payoff P.

    In the given dataset:
    A, D-A are an integer timestamp;
    L is composed by x, y float values;
    P is a float value.
    """

    def __init__(self, arrival_time: int, location: Tuple[float], deadline: int, payoff: float) -> None:
        """Initializes a Task object. """
        super().__init__(arrival_time, location, deadline, capacity=1)
        self.payoff = payoff

    def __str__(self) -> str:
        """Returns a string representation of the task."""
        return f"t: loc = ({self.location[0]:.2f}, {self.location[1]:.2f}), time = ({self.arrival_time}, {self.deadline}), payoff = {self.payoff:.2f}"

    def utility_with(self, worker: type[Worker]) -> float:
        """The utility that a worker performs a task is measured by P_t * Delta_w.
        Without loss of generality, we will assume that utility is between 0 and U_max.
        """
        return self.payoff * worker.success_ratio

    def can_be_with(self, worker: type[Worker]) -> bool:
        """Check the 4 constraints, namely:
        Invariable Constraint, Range Constraint, Deadline Constraint, Capacity Constraint
        """
        if self.flow > 0:
            return False

        if worker.distance_from(self) > worker.radius:
            return False

        if worker.arrival_time >= self.deadline or self.arrival_time >= worker.deadline:
            return False

        if worker.capacity <= worker.flow or self.capacity <= self.flow:
            return False

        return True

Based on different assumptions on the arrival order, we introduce two competitive analysis models for the GOMA problem, the adversarial
order model and the random order model, which focus on the
worst-case arrival order and the stochastic arrival order of
all the tasks and workers, respectively.

In [89]:
class GOMA:
    """Given a set of tasks T, a set of workers W, and a utility function U (., .),
    the GOMA problem is to find an allocation M 
    to maximize the total utility such that the following constraints are satisfied:
    1. Deadline Constraint;
    2. Invariable Constraint;
    3. Capacity Constraint;
    4. Range Constraint.
    """

    def __init__(self, filepath: str = "../dataset/real/EverySender_cap1/data_00.txt"):
        assert os.path.exists(filepath)
        self.filepath = filepath
        self.workers: list[Worker] = []
        self.tasks: list[Task] = []

        with open(file=self.filepath, mode='r') as fin:
            pieces = fin.readline().strip().split()
            self.num_worker = int(pieces[0])
            self.num_task = int(pieces[1])
            self.max_utility = float(pieces[2])
            self.sum_capacity = int(pieces[3])

    def __iter__(self):
        """
        Iterates over the vertices (mimic the online scenario).

        Yields:
            Task: A task with attributes arrival_time, location, deadline, payoff, and radius.
            Worker: A worker with attributes arrival_time, location, capacity, deadline, and success_ratio.
        """
        with open(file=self.filepath, mode='r') as fin:
            # skip the header
            pieces = fin.readline().strip().split()

            for _ in range(self.num_worker + self.num_task):
                pieces = fin.readline().strip().split()
                args = {"arrival_time": int(pieces[0]),
                        "location": (float(pieces[2]), float(pieces[3]))}
                match pieces[1]:
                    case 't':
                        args["deadline"] = int(
                            pieces[4]) + args["arrival_time"]
                        args["payoff"] = float(pieces[5])
                        task = Task(**args)
                        print(task)
                        self.tasks.append(task)
                        yield task
                    case 'w':
                        args["radius"] = float(pieces[4])
                        args["capacity"] = int(pieces[5])
                        args["deadline"] = int(
                            pieces[6]) + args["arrival_time"]
                        args["success_ratio"] = float(pieces[7])
                        worker = Worker(**args)
                        print(worker)
                        self.workers.append(worker)
                        yield worker
                    case _:
                        raise StopIteration

# Baseline Algorithm: Extended Greedy-RT

**Basic Idea**: first randomly choose a threshold on the weights of edges, 

and then randomly choose an edge incident to each newly arrived vertex among those edges whose weights are *no less than* the threshold.

In [90]:
class Algo1:
    def __init__(self, problem: type[GOMA]) -> None:
        self.problem = problem
        self.total_utility = 0

    def select_threshold(self):
        """Step 1: 
        theta := ceil(ln(U_max + 1))
            k := random drawn from {0, ..., theta - 1}
        """
        theta = int(math.ceil(math.log(self.problem.max_utility+1)))
        # n.b. for analysis, you may need to look at every possible value of k
        k = random.randint(0, theta-1)
        print("DEBUG: ", f"theta={theta}, k={k}")
        self.threshold = pow(math.e, k)

    def append_to_match(self, u, v):
        """M := M \cup {(u, v)}"""
        self.total_utility += u.utility_with(v)
        u.flow += 1
        v.flow += 1

    def try_for_new_vertex(self, v: type[Vertex]):
        """Step 2 within the loop:
        foreach newly arrived vertex v dp
            cand := {u|u satisfies all the constraints and U(u, v) >= e^k}
            if cand is not empty then
                u* := an arbitrary item is chosen from cand
                m.append((u*, v))
        """
        assert isinstance(v, Worker) or isinstance(v, Task)

        candidates = filter(
            lambda u:  u.can_be_with(
                v) and u.utility_with(v) >= self.threshold,
            self.problem.tasks if isinstance(
                v, Worker) else self.problem.workers
        )

        u = None
        for cand in candidates:
            # n.b filter() returns an iterator yielding those items
            u = cand
            break

        if u:
            self.append_to_match(u, v)

    def run(self):
        """
        1. select_threshold: randomly choose a threshold (ek) on the weights of edges according to the estimated maximum weight Umax;
        2. try_for_new_vertex: When a new vertex arrives, adds an edge among the ones whose weights are no less than a threshold and that satisfy all the constraints to the result;
        """

        self.select_threshold()
        for v in self.problem:
            self.try_for_new_vertex(v)
        print(self.threshold, self.total_utility)

In [91]:
problem = GOMA(
    filepath="../dataset/real/EverySender_cap1/data_00.txt")
solver = Algo1(problem=problem)
solver.run()

DEBUG:  theta=3, k=2
t: loc = (1.79, 1.73), time = (74875, 75475), payoff = 3.6
t: loc = (3.90, 3.62), time = (282944, 283544), payoff = 4.2
t: loc = (1.50, 1.50), time = (433979, 434579), payoff = 9.0
t: loc = (1.43, 2.34), time = (446086, 446686), payoff = 6.9
t: loc = (3.78, 0.34), time = (485259, 485859), payoff = 3.1
t: loc = (3.39, 2.22), time = (406875, 407475), payoff = 8.5
t: loc = (4.56, 4.91), time = (226296, 226896), payoff = 5.3
w: loc = (1.23, 1.01), rad = 1.00, cap = 1, time = (444046, 444646), success_ratio = 0.598
w: loc = (1.01, 0.20), rad = 1.00, cap = 1, time = (219014, 219614), success_ratio = 0.461
t: loc = (2.78, 0.78), time = (43327, 43927), payoff = 5.7
t: loc = (2.45, 3.82), time = (469359, 469959), payoff = 9.8
t: loc = (3.75, 0.03), time = (257234, 257834), payoff = 8.1
t: loc = (1.85, 1.38), time = (200965, 201565), payoff = 3.4
t: loc = (4.84, 4.37), time = (109147, 109747), payoff = 3.9
t: loc = (1.16, 2.55), time = (405303, 405903), payoff = 1.9
t: loc =